In [1]:
import pandas as pd
import spacy
from spacy.tokenizer import Tokenizer
from spacy.symbols import ORTH

import data_preparation as mdp
from mercari_config import MercariConfig

In [2]:
%%time

nlp = spacy.load('en_core_web_md', disable=['parser', 'tagger'])  
tokenizer = Tokenizer(nlp.vocab)

train_data = mdp.load_data(MercariConfig.TRAINING_SET_PREP_FILE)

CPU times: user 19.6 s, sys: 2.06 s, total: 21.7 s
Wall time: 20.4 s


In [3]:
nlp.pipeline

[('ner', <spacy.pipeline.EntityRecognizer at 0x7efc27473fc0>)]

In [4]:
nlp.tokenizer.add_special_case('[rm]', [{ORTH: '[rm]'}])

In [5]:
%%time

word = mdp.init_word_dict()
max_words_in_name = mdp.walk_items(items=train_data['name'], nlp=nlp, word=word)

CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 36.4 ms


In [6]:
word2index = mdp.build_word2index(word=word)

In [7]:
word2index

,word_id,entity_type,count
word,,,
___PAD___,0,___NONE_ENTITY___,0
___START___,1,___NONE_ENTITY___,0
___OOV___,2,___NONE_ENTITY___,0
[rm],3,___NONE_ENTITY___,0
___VERY_EMPTY_NAME___,4,___NONE_ENTITY___,0
___VERY_EMPTY_CATEGORY___,5,___NONE_ENTITY___,0
___VERY_EMPTY_BRAND___,6,___NONE_ENTITY___,0
___VERY_EMPTY_DESCRIPTION___,7,___NONE_ENTITY___,0
's,8,___NONE_ENTITY___,1


In [8]:
mdp.save_word2index(word2index=word2index, file_name=MercariConfig.WORD_2_INDEX_4_NAME_FILE )

In [9]:
len(word2index)

34

In [10]:
word2index.count()

word_id        34
entity_type    34
count          34
dtype: int64

In [11]:
max_words_in_name

5

In [12]:
word2index.groupby("entity_type").count()

,word_id,count
entity_type,,
CARDINAL,3,3
LOC,1,1
ORG,5,5
PERSON,1,1
___NONE_ENTITY___,24,24
